## **1. NaiveBayes Classifier**
1. 주어진 데이터를 전처리합니다.
2. NaiveBayes 분류기 모델을 구현하고 학습 데이터로 이를 학습시킵니다.
3. 간단한 test case로 결과를 확인합니다.

### **필요 패키지 import**

In [1]:
!pip install konlpy # 한국어 토크나이징을 위한 패키지

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 10.8MB/s 
     |████████████████████████████████| 460kB 54.7MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [4]:
from tqdm import tqdm

# 다양한 한국어 형태소 분석기가 클래스로 구현되어 있음
from konlpy import tag 

from collections import defaultdict

import math

### **학습 및 테스트 데이터 전처리**

Sample 데이터를 확인합니다.  
긍정($1$), 부정($0$) 2가지 class로 구성되어 있습니다.

In [5]:
# 레스토랑 리뷰데이터로 각 문장은 긍정 / 부정으로 분류되어 있음
train_data = [
  "정말 맛있습니다. 추천합니다.",
  "기대했던 것보단 별로였네요.",
  "다 좋은데 가격이 너무 비싸서 다시 가고 싶다는 생각이 안 드네요.",
  "완전 최고입니다! 재방문 의사 있습니다.",
  "음식도 서비스도 다 만족스러웠습니다.",
  "위생 상태가 좀 별로였습니다. 좀 더 개선되기를 바랍니다.",
  "맛도 좋았고 직원분들 서비스도 너무 친절했습니다.",
  "기념일에 방문했는데 음식도 분위기도 서비스도 다 좋았습니다.",
  "전반적으로 음식이 너무 짰습니다. 저는 별로였네요.",
  "위생에 조금 더 신경 썼으면 좋겠습니다. 조금 불쾌했습니다."
]
train_labels = [1, 0, 0, 1, 1, 0, 1, 1, 0, 0]

test_data = [
  "정말 좋았습니다. 또 가고 싶네요.",
  "별로였습니다. 되도록 가지 마세요.",
  "다른 분들께도 추천드릴 수 있을 만큼 만족했습니다.",
  "서비스가 좀 더 개선되었으면 좋겠습니다. 기분이 좀 나빴습니다."
]

KoNLPy 패키지에서 제공하는 Twitter(Okt) tokenizer를 사용하여 tokenization합니다.

In [6]:
tokenizer = tag.Okt()

In [8]:
def make_tokenized(data):
    tokenized = []  # 단어 단위로 나뉜 리뷰 데이터.

    for sent in tqdm(data):
        tokens = tokenizer.morphs(sent) # 문장 -> 토큰 리스트로 바꿔주는 메소드
        tokenized.append(tokens)
    return tokenized

In [9]:
# 토큰 == 단어가 아님
# 토큰이란 보통 하나의 의미를 이루는 모음이라고 생각하는 것이 보다 맞음
# 문장을 토크나이징하는 다양한 방법이 존재함
train_tokenized = make_tokenized(train_data)
test_tokenized = make_tokenized(test_data)

100%|██████████| 4/4 [00:00<00:00, 86.37it/s]


In [10]:
train_tokenized

[['정말', '맛있습니다', '.', '추천', '합니다', '.'],
 ['기대했던', '것', '보단', '별로', '였네요', '.'],
 ['다',
  '좋은데',
  '가격',
  '이',
  '너무',
  '비싸서',
  '다시',
  '가고',
  '싶다는',
  '생각',
  '이',
  '안',
  '드네',
  '요',
  '.'],
 ['완전', '최고', '입니다', '!', '재', '방문', '의사', '있습니다', '.'],
 ['음식', '도', '서비스', '도', '다', '만족스러웠습니다', '.'],
 ['위생',
  '상태',
  '가',
  '좀',
  '별로',
  '였습니다',
  '.',
  '좀',
  '더',
  '개선',
  '되',
  '기를',
  '바랍니다',
  '.'],
 ['맛', '도', '좋았고', '직원', '분들', '서비스', '도', '너무', '친절했습니다', '.'],
 ['기념일',
  '에',
  '방문',
  '했는데',
  '음식',
  '도',
  '분위기',
  '도',
  '서비스',
  '도',
  '다',
  '좋았습니다',
  '.'],
 ['전반', '적', '으로', '음식', '이', '너무', '짰습니다', '.', '저', '는', '별로', '였네요', '.'],
 ['위생', '에', '조금', '더', '신경', '썼으면', '좋겠습니다', '.', '조금', '불쾌했습니다', '.']]

학습데이터 기준으로 가장 많이 등장한 단어부터 순서대로 vocab에 추가합니다.

In [11]:
word_count = defaultdict(int)  # Key: 단어, Value: 등장 횟수

for tokens in tqdm(train_tokenized):
  for token in tokens:
    word_count[token] += 1

100%|██████████| 10/10 [00:00<00:00, 15169.27it/s]


In [12]:
word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
print(len(word_count))

66


In [18]:
word_count[:5]

[('.', 14), ('도', 7), ('별로', 3), ('다', 3), ('이', 3)]

In [14]:
w2i = {}  # Key: 단어, Value: 단어의 index
for pair in tqdm(word_count):
  if pair[0] not in w2i:
    w2i[pair[0]] = len(w2i)

100%|██████████| 66/66 [00:00<00:00, 103718.27it/s]


In [15]:
len(w2i)

66

In [16]:
# word to index
w2i

{'!': 35,
 '.': 0,
 '가': 41,
 '가격': 23,
 '가고': 26,
 '개선': 43,
 '것': 20,
 '기념일': 52,
 '기대했던': 19,
 '기를': 45,
 '너무': 5,
 '는': 61,
 '다': 3,
 '다시': 25,
 '더': 12,
 '도': 1,
 '되': 44,
 '드네': 30,
 '만족스러웠습니다': 39,
 '맛': 47,
 '맛있습니다': 16,
 '바랍니다': 46,
 '방문': 9,
 '별로': 2,
 '보단': 21,
 '분들': 50,
 '분위기': 54,
 '불쾌했습니다': 65,
 '비싸서': 24,
 '상태': 40,
 '생각': 28,
 '서비스': 7,
 '신경': 62,
 '싶다는': 27,
 '썼으면': 63,
 '안': 29,
 '에': 13,
 '였네요': 8,
 '였습니다': 42,
 '완전': 32,
 '요': 31,
 '위생': 10,
 '으로': 58,
 '음식': 6,
 '의사': 37,
 '이': 4,
 '입니다': 34,
 '있습니다': 38,
 '재': 36,
 '저': 60,
 '적': 57,
 '전반': 56,
 '정말': 15,
 '조금': 14,
 '좀': 11,
 '좋겠습니다': 64,
 '좋았고': 48,
 '좋았습니다': 55,
 '좋은데': 22,
 '직원': 49,
 '짰습니다': 59,
 '최고': 33,
 '추천': 17,
 '친절했습니다': 51,
 '합니다': 18,
 '했는데': 53}

### **모델 Class 구현**

NaiveBayes Classifier 모델 클래스를 구현합니다.

*   `self.k`: Smoothing을 위한 상수.
*   `self.w2i`: 사전에 구한 vocab.
*   `self.priors`: 각 class의 prior 확률.
*   `self.likelihoods`: 각 token의 특정 class 조건 내에서의 likelihood.


In [19]:
class NaiveBayesClassifier():
  def __init__(self, w2i, k=0.1):
    self.k = k # 곱해지는 조건부확률이 0이 됨을 방지하기 위한 bias
    self.w2i = w2i
    self.priors = {} # 해당 label 개수 / 전체 label 개수
    self.likelihoods = {} # 각 라벨별 토큰별 전체 단어 중 해당 

  def train(self, train_tokenized, train_labels):
    self.set_priors(train_labels)  # Priors 계산.
    self.set_likelihoods(train_tokenized, train_labels)  # Likelihoods 계산.

  def inference(self, tokens):
    log_prob0 = 0.0 # 극단적으로 작아는 곱을 방지하기 위한 log 덧셈으로 치환
    log_prob1 = 0.0

    for token in tokens:
      if token in self.likelihoods:  # 학습 당시 추가했던 단어에 대해서만 고려.
        log_prob0 += math.log(self.likelihoods[token][0])
        log_prob1 += math.log(self.likelihoods[token][1])

    # 마지막에 prior를 고려.
    log_prob0 += math.log(self.priors[0])
    log_prob1 += math.log(self.priors[1])

    if log_prob0 >= log_prob1:
      return 0
    else:
      return 1

  def set_priors(self, train_labels):
    class_counts = defaultdict(int)
    for label in tqdm(train_labels):
      class_counts[label] += 1
    
    for label, count in class_counts.items():
      self.priors[label] = class_counts[label] / len(train_labels)

  def set_likelihoods(self, train_tokenized, train_labels):
    token_dists = {}  # 각 단어의 특정 class 조건 하에서의 등장 횟수.
    class_counts = defaultdict(int)  # 특정 class에서 등장한 모든 단어의 등장 횟수.

    for i, label in enumerate(tqdm(train_labels)):
      count = 0
      for token in train_tokenized[i]:
        if token in self.w2i:  # 학습 데이터로 구축한 vocab에 있는 token만 고려.
          if token not in token_dists:
            token_dists[token] = {0:0, 1:0}
          token_dists[token][label] += 1
          count += 1
      class_counts[label] += count

    for token, dist in tqdm(token_dists.items()):
      if token not in self.likelihoods:
        self.likelihoods[token] = {
            0:(token_dists[token][0] + self.k) / (class_counts[0] + len(self.w2i)*self.k),
            1:(token_dists[token][1] + self.k) / (class_counts[1] + len(self.w2i)*self.k),
        }

### **모델 학습**

모델 객체를 만들고 학습 데이터로 학습시킵니다.

In [23]:
train_labels

[1, 0, 0, 1, 1, 0, 1, 1, 0, 0]

In [20]:
classifier = NaiveBayesClassifier(w2i)
classifier.train(train_tokenized, train_labels)

100%|██████████| 66/66 [00:00<00:00, 67765.99it/s]


### **테스트**

Test sample에 대한 결과는 다음과 같습니다.

In [21]:
preds = []
for test_tokens in tqdm(test_tokenized):
  pred = classifier.inference(test_tokens)
  preds.append(pred)

100%|██████████| 4/4 [00:00<00:00, 5054.90it/s]


In [22]:
preds

[1, 0, 1, 0]

### **코드 분석**

In [24]:
def set_priors(self, train_labels):
    class_counts = defaultdict(int)
    for label in tqdm(train_labels):
      class_counts[label] += 1
    
    for label, count in class_counts.items():
      self.priors[label] = class_counts[label] / len(train_labels)

In [25]:
train_labels

[1, 0, 0, 1, 1, 0, 1, 1, 0, 0]

In [26]:
class_counts = defaultdict(int)

for label in tqdm(train_labels):
    class_counts[label] += 1

class_counts

100%|██████████| 10/10 [00:00<00:00, 30437.62it/s]


defaultdict(int, {0: 5, 1: 5})

In [27]:
priors = {}
for label, count in class_counts.items():
    priors[label] = class_counts[label] / len(train_labels)

In [28]:
priors

{0: 0.5, 1: 0.5}

In [33]:
token_dists = {}
class_counts = defaultdict(int)

for i, label in enumerate(tqdm(train_labels)):
    count = 0
    for token in train_tokenized[i]:
        if token in w2i:
            if token not in token_dists:
                token_dists[token] = {0:0, 1:0}
            token_dists[token][label] += 1
            count += 1
        class_counts[label] += count

100%|██████████| 10/10 [00:00<00:00, 2496.76it/s]


In [38]:
likelihoods = {}
k = 0.1

for token, dist in tqdm(token_dists.items()):
    if token not in likelihoods:
        likelihoods[token] = {
            0:(token_dists[token][0] + k) / (class_counts[0] + len(w2i)*k),
            1:(token_dists[token][1] + k) / (class_counts[1] + len(w2i)*k),
        }

100%|██████████| 66/66 [00:00<00:00, 18200.14it/s]


In [39]:
likelihoods

{'!': {0: 0.000244140625, 1: 0.004460665044606651},
 '.': {0: 0.019775390624999997, 1: 0.02473641524736415},
 '가': {0: 0.002685546875, 1: 0.00040551500405515005},
 '가격': {0: 0.002685546875, 1: 0.00040551500405515005},
 '가고': {0: 0.002685546875, 1: 0.00040551500405515005},
 '개선': {0: 0.002685546875, 1: 0.00040551500405515005},
 '것': {0: 0.002685546875, 1: 0.00040551500405515005},
 '기념일': {0: 0.000244140625, 1: 0.004460665044606651},
 '기대했던': {0: 0.002685546875, 1: 0.00040551500405515005},
 '기를': {0: 0.002685546875, 1: 0.00040551500405515005},
 '너무': {0: 0.005126953125, 1: 0.004460665044606651},
 '는': {0: 0.002685546875, 1: 0.00040551500405515005},
 '다': {0: 0.002685546875, 1: 0.00851581508515815},
 '다시': {0: 0.002685546875, 1: 0.00040551500405515005},
 '더': {0: 0.005126953125, 1: 0.00040551500405515005},
 '도': {0: 0.000244140625, 1: 0.02879156528791565},
 '되': {0: 0.002685546875, 1: 0.00040551500405515005},
 '드네': {0: 0.002685546875, 1: 0.00040551500405515005},
 '만족스러웠습니다': {0: 0.000244